In [1]:
%load_ext autoreload
%autoreload 2

from cnn import CNN
from dyn_cnn import DynCNN
from util import Util
from patch import Patch
from fixer import Fixer
from uglify import Uglify

import cPickle as pickle
import os
import mahotas as mh
import numpy as np
import time

import mlproof as mlp


from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/nolearn/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
tifffile.py:1995: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


In [2]:
from sklearn.metrics import classification_report, accuracy_score


In [5]:
with open('/home/d/nets/MergeNet_cylinder1_larger_border_overlap_usevars/net.p', 'rb') as f:
    cnn_large_border = pickle.load(f)
with open('/home/d/nets/MergeNet_cylinder1_border_overlap_usevars/net.p', 'rb') as f:
    cnn_small_border = pickle.load(f)
with open('/home/d/nets/MergeNet_cylinder1_nooverlap_usevars/net.p', 'rb') as f:
    cnn_no_border = pickle.load(f)

In [6]:
X_train, y_train, X_test, y_test = mlp.Patch.load('cylinder1')

Loaded /home/d/patches//cylinder1/ in 0.000701904296875 seconds.


In [19]:
cnn_large_border.layers.

[('image_input', lasagne.layers.input.InputLayer),
 ('image_conv1', lasagne.layers.conv.Conv2DLayer),
 ('image_pool1', lasagne.layers.pool.MaxPool2DLayer),
 ('image_conv2', lasagne.layers.conv.Conv2DLayer),
 ('image_pool2', lasagne.layers.pool.MaxPool2DLayer),
 ('prob_input', lasagne.layers.input.InputLayer),
 ('prob_conv1', lasagne.layers.conv.Conv2DLayer),
 ('prob_pool1', lasagne.layers.pool.MaxPool2DLayer),
 ('prob_conv2', lasagne.layers.conv.Conv2DLayer),
 ('prob_pool2', lasagne.layers.pool.MaxPool2DLayer),
 ('binary_input', lasagne.layers.input.InputLayer),
 ('binary_conv1', lasagne.layers.conv.Conv2DLayer),
 ('binary_pool1', lasagne.layers.pool.MaxPool2DLayer),
 ('binary_conv2', lasagne.layers.conv.Conv2DLayer),
 ('binary_pool2', lasagne.layers.pool.MaxPool2DLayer),
 ('border_input', lasagne.layers.input.InputLayer),
 ('border_conv1', lasagne.layers.conv.Conv2DLayer),
 ('border_pool1', lasagne.layers.pool.MaxPool2DLayer),
 ('border_conv2', lasagne.layers.conv.Conv2DLayer),
 ('bor

In [7]:
cnn_no_border_y_true, cnn_no_border_y_pred = y_test, cnn_no_border.predict({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array']})

In [11]:
print(classification_report(cnn_no_border_y_true, cnn_no_border_y_pred))


             precision    recall  f1-score   support

          0       0.91      0.89      0.90     13408
          1       0.89      0.92      0.90     13408

avg / total       0.90      0.90      0.90     26816



In [15]:
cnn_no_border.score({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array']},
           y_test)

0.9034158711217184

In [9]:
cnn_large_border_y_true, cnn_large_border_y_pred = y_test, cnn_large_border.predict({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array'],'border_input':X_test['larger_border_overlap']})

In [12]:
print(classification_report(cnn_large_border_y_true, cnn_large_border_y_pred))


             precision    recall  f1-score   support

          0       0.89      0.90      0.90     13408
          1       0.90      0.89      0.89     13408

avg / total       0.90      0.90      0.90     26816



In [16]:
cnn_large_border.score({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array'],
           'border_input':X_test['larger_border_overlap']},
           y_test)

0.8952863961813843

In [10]:
cnn_small_border_y_true, cnn_small_border_y_pred = y_test, cnn_small_border.predict({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array'],'border_input':X_test['border_overlap']})

In [13]:
print(classification_report(cnn_small_border_y_true, cnn_small_border_y_pred))


             precision    recall  f1-score   support

          0       0.90      0.91      0.90     13408
          1       0.91      0.90      0.90     13408

avg / total       0.90      0.90      0.90     26816



In [17]:
cnn_small_border.score({'image_input': X_test['image'],
           'prob_input': X_test['prob'],
           'binary_input': X_test['merged_array'],
           'border_input':X_test['border_overlap']},
           y_test)

0.9018869331742243